In [1]:
import json
import spotipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyClientCredentials
from math import pi

client_id = 'acc540a7f8844bd887f02c72b9508732' #insert your client id
client_secret = '183b3d48db27418a8e6b2b73e14f7e53' # insert your client secret id here

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_id='spotify:playlist:37i9dQZF1EjqdQrvqcMFqb' #insert your playlist id
results = sp.playlist(playlist_id)


In [2]:
# create a list of song ids
ids=[]

for item in results['tracks']['items']:
        track = item['track']['id']
        ids.append(track)
        
song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

for song_id in ids:
    # get song meta data
    meta = sp.track(song_id)
    
    # song id
    song_meta['id'].append(song_id)

    # album name
    album=meta['album']['name']
    song_meta['album']+=[album]

    # song name
    song=meta['name']
    song_meta['name']+=[song]
    
    # artist name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

song_meta_df=pd.DataFrame.from_dict(song_meta)

# check the song feature
features = sp.audio_features(song_meta['id'])
# change dictionary to dataframe
features_df=pd.DataFrame.from_dict(features)

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000

# combine two dataframe
final_df=song_meta_df.merge(features_df)
   

In [3]:
final_df

,id,album,name,artist,explicit,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4DdoBfBEJ2gr6rUlRT34wz,Red Forest,Red Forest,CloZee,False,0,0.629,0.746,7,-4.899,...,0.651000,0.1710,0.3020,100.004,audio_features,spotify:track:4DdoBfBEJ2gr6rUlRT34wz,https://api.spotify.com/v1/tracks/4DdoBfBEJ2gr...,https://api.spotify.com/v1/audio-analysis/4Ddo...,3.555000,4
1,7Jcf74UJvImsHrGOqSS0tG,it's not us,You & You Alone,Umphrey's McGee,False,45,0.514,0.282,0,-13.793,...,0.000021,0.1140,0.1170,89.942,audio_features,spotify:track:7Jcf74UJvImsHrGOqSS0tG,https://api.spotify.com/v1/tracks/7Jcf74UJvIms...,https://api.spotify.com/v1/audio-analysis/7Jcf...,3.979367,3
2,2jt2WxXMCD4zjACthkJQVE,Kamikaze,The Ringer,Eminem,True,72,0.939,0.551,1,-9.126,...,0.000000,0.1920,0.4200,118.552,audio_features,spotify:track:2jt2WxXMCD4zjACthkJQVE,https://api.spotify.com/v1/tracks/2jt2WxXMCD4z...,https://api.spotify.com/v1/audio-analysis/2jt2...,5.632350,4
3,1Jc15fxurf0RSLfiZoslaY,Inner G,Inner G (feat. Ganavya),"Zebbler Encanti Experience, Ganavya",False,37,0.618,0.825,1,-5.711,...,0.712000,0.1030,0.0467,159.998,audio_features,spotify:track:1Jc15fxurf0RSLfiZoslaY,https://api.spotify.com/v1/tracks/1Jc15fxurf0R...,https://api.spotify.com/v1/audio-analysis/1Jc1...,4.200000,4
4,5cio8CvmB7nBA1OxiSYqrg,Visions of Backbeat,Midnight on the Run,BoomBox,False,44,0.823,0.455,10,-9.245,...,0.356000,0.1240,0.1560,102.024,audio_features,spotify:track:5cio8CvmB7nBA1OxiSYqrg,https://api.spotify.com/v1/tracks/5cio8CvmB7nB...,https://api.spotify.com/v1/audio-analysis/5cio...,9.986667,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6veGXXMZLNEVYyB8bipmAv,NVSB Remixes,So Butterfly (2014 Version) [D.V.S* Remix],"Bassnectar, D.V.S*",False,0,0.584,0.883,10,-3.176,...,0.331000,0.0863,0.0386,104.993,audio_features,spotify:track:6veGXXMZLNEVYyB8bipmAv,https://api.spotify.com/v1/tracks/6veGXXMZLNEV...,https://api.spotify.com/v1/audio-analysis/6veG...,4.277083,4
96,4zD13A370cQQIkxQxa17Gp,ZONKEY,Sad Clint Eastwood,Umphrey's McGee,False,43,0.710,0.826,2,-6.963,...,0.005070,0.0687,0.4910,92.989,audio_features,spotify:track:4zD13A370cQQIkxQxa17Gp,https://api.spotify.com/v1/tracks/4zD13A370cQQ...,https://api.spotify.com/v1/audio-analysis/4zD1...,4.383667,4
97,5H0dVldtI6HJZnEXzigKfY,It's Dark And Hell Is Hot,Ruff Ryders' Anthem,DMX,True,0,0.879,0.573,10,-8.113,...,0.000000,0.2560,0.7300,91.037,audio_features,spotify:track:5H0dVldtI6HJZnEXzigKfY,https://api.spotify.com/v1/tracks/5H0dVldtI6HJ...,https://api.spotify.com/v1/audio-analysis/5H0d...,3.574000,4
98,4XpQ2F8NtzWMZ3g5t8qv2Y,Tallahassee,No Children,The Mountain Goats,False,47,0.606,0.411,1,-7.566,...,0.000649,0.1030,0.4120,84.944,audio_features,spotify:track:4XpQ2F8NtzWMZ3g5t8qv2Y,https://api.spotify.com/v1/tracks/4XpQ2F8NtzWM...,https://api.spotify.com/v1/audio-analysis/4XpQ...,2.805117,4


In [4]:
music_feature = features_df[['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']]

In [5]:
music_feature.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.531992,0.63650,-9.067930,0.177718,0.292319,0.256898,0.286347,0.361723,117.509360,5.142458
std,0.204937,0.25043,4.935737,0.263158,0.327186,0.350940,0.277784,0.227727,31.352656,2.616950
min,0.086200,0.03210,-32.418000,0.026800,0.000027,0.000000,0.022300,0.027900,50.324000,1.925333
25%,0.382250,0.46550,-11.367750,0.038650,0.018450,0.000009,0.105250,0.187000,92.401000,3.645775
50%,0.524000,0.67950,-8.199500,0.057300,0.129000,0.006100,0.128500,0.338500,116.641500,4.395500
75%,0.666000,0.84950,-5.327000,0.161750,0.604250,0.651500,0.375250,0.556500,138.193500,5.638908
max,0.939000,0.97900,-1.803000,0.963000,0.986000,0.956000,0.989000,0.922000,191.994000,21.651783


In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
scaler = MinMaxScaler()

In [8]:
df_scaled = pd.DataFrame(scaler.fit_transform(music_feature), columns=music_feature.columns)

In [9]:
df_scaled

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.636492,0.753934,0.898873,0.021897,0.000358,0.680962,0.153822,0.306565,0.350674,0.082613
1,0.501642,0.263914,0.608362,0.003632,0.450289,0.000022,0.094859,0.099653,0.279650,0.104126
2,1.000000,0.547999,0.760804,0.282205,0.031515,0.000000,0.175546,0.438542,0.481598,0.187921
3,0.623593,0.837364,0.872350,0.044755,0.077865,0.744770,0.083480,0.021027,0.774151,0.115310
4,0.863977,0.446615,0.756917,0.045824,0.060927,0.372385,0.105203,0.143273,0.364933,0.408656
...,...,...,...,...,...,...,...,...,...,...
95,0.583724,0.898617,0.955153,0.008759,0.004759,0.346234,0.066205,0.011967,0.385890,0.119218
96,0.731473,0.838420,0.831455,0.019013,0.002701,0.005303,0.047998,0.517951,0.301158,0.124621
97,0.929644,0.571232,0.793892,0.297159,0.064477,0.000000,0.241750,0.785259,0.287379,0.083576
98,0.609522,0.400148,0.811759,0.015488,0.608508,0.000679,0.083480,0.429594,0.244371,0.044599


In [10]:
# convert column names into a list
categories=list(music_feature.columns)
# number of categories
N=len(categories)

In [11]:
# create a list with the average of all features
value=list(music_feature.mean())

In [12]:
engine = create_engine('sqlite://', echo=False)

In [13]:
df_scaled.to_sql('users', con=engine)

engine.execute("SELECT * FROM users").fetchall()

[(0, 0.6364915572232646, 0.7539338895342699, 0.8988731014208722, 0.02189703054902799, 0.00035771781510786257, 0.6809623430962343, 0.1538222819902762, 0.30656526115647015, 0.35067410178584035, 0.08261327642158964),
 (1, 0.5016416510318948, 0.2639138240574506, 0.6083619140943982, 0.0036317026276436702, 0.45028903944297854, 2.1757322175732217e-05, 0.0948587979724837, 0.09965328263057824, 0.2796498905908096, 0.10412584795203056),
 (2, 0.9999999999999998, 0.5479987327067274, 0.7608035276825087, 0.282204657124546, 0.031514767092435725, 0.0, 0.17554567083893657, 0.4385415501621742, 0.48159808004517535, 0.1879211245138718),
 (3, 0.6235928705440901, 0.8373640299926075, 0.872350155152703, 0.04475539414654987, 0.07786493479991889, 0.7447698744769874, 0.08347988000413778, 0.021026730790739287, 0.7741511964424364, 0.11531049259581258),
 (4, 0.8639774859287055, 0.4466152708839371, 0.7569165441776907, 0.045823541978209786, 0.06092734616282987, 0.3723849372384937, 0.10520326885279818, 0.14327256459009

In [14]:
df_scaled.to_csv('DF4.csv', encoding='utf-8', index='False')

In [15]:
playlist_id='spotify:playlist:37i9dQZF1Et8H9DlUAsrMD' #insert your playlist id
results = sp.playlist(playlist_id)

In [16]:
# create a list of song ids
ids=[]

for item in results['tracks']['items']:
        track = item['track']['id']
        ids.append(track)
        
song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

for song_id in ids:
    # get song's meta data
    meta = sp.track(song_id)
    
    # song id
    song_meta['id'].append(song_id)

    # album name
    album=meta['album']['name']
    song_meta['album']+=[album]

    # song name
    song=meta['name']
    song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

song_meta_df=pd.DataFrame.from_dict(song_meta)

# check the song feature
features = sp.audio_features(song_meta['id'])
# change dictionary to dataframe
features_df=pd.DataFrame.from_dict(features)

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000

# combine two dataframe
final_df=song_meta_df.merge(features_df)

In [17]:
scaler = MinMaxScaler()

In [18]:
df_scaled = pd.DataFrame(scaler.fit_transform(music_feature), columns=music_feature.columns)

In [19]:
# convert column names into a list
categories=list(music_feature.columns)
# number of categories
N=len(categories)

In [20]:
# create a list with the average of all features
value=list(music_feature.mean())

In [21]:
df_scaled.to_csv('DF5.csv', encoding='utf-8', index='False')

In [22]:
playlist_id='spotify:playlist:37i9dQZF1EM6SkSK3apweL' #insert your playlist id
results = sp.playlist(playlist_id)

In [23]:
# create a list of song ids
ids=[]

for item in results['tracks']['items']:
        track = item['track']['id']
        ids.append(track)
        
song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

for song_id in ids:
    # get song's meta data
    meta = sp.track(song_id)
    
    # song id
    song_meta['id'].append(song_id)

    # album name
    album=meta['album']['name']
    song_meta['album']+=[album]

    # song name
    song=meta['name']
    song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

song_meta_df=pd.DataFrame.from_dict(song_meta)

# check the song feature
features = sp.audio_features(song_meta['id'])
# change dictionary to dataframe
features_df=pd.DataFrame.from_dict(features)

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000

# combine two dataframe
final_df=song_meta_df.merge(features_df)

In [24]:
scaler = MinMaxScaler()

In [25]:
df_scaled = pd.DataFrame(scaler.fit_transform(music_feature), columns=music_feature.columns)

In [26]:
# convert column names into a list
categories=list(music_feature.columns)
# number of categories
N=len(categories)

In [27]:
# create a list with the average of all features
value=list(music_feature.mean())

In [28]:
df_scaled.to_csv('DF6.csv', encoding='utf-8', index='False')

In [32]:
fig = px.scatter_3d(df_scaled, x="valence", y="energy", z="acousticness", color="danceability", 
                    symbol="acousticness", width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()